In [82]:
import os
from collections import defaultdict
vocabulary = defaultdict(list)
category_frequency = defaultdict(float)
total_documents = 0
test_df = defaultdict(list)



def classify():
    global total_documents

    for directory in os.listdir('./'):
        ctr = 0
        if os.path.isdir(directory):
        
            category = directory
            length = len(os.listdir(directory))
            

            for filename in os.listdir(directory):
                
                # Check if the file is a regular file (not a directory)
                if os.path.isfile(os.path.join(directory, filename)):
                    
                    with open(os.path.join(directory, filename), 'r') as file:
                        
                        words = file.read().split()

                        # store the two-adjacent phrases
                        new_doc = [] 
                        for i in range(len(words)-1):
                            phrase = f'{words[i]} {words[i+1]}'
                            new_doc.append(phrase)

                        
                        if ctr < 0.8*(length):
                            for word in new_doc:
                                vocabulary[category].append(word)
                                
                        else:
                            test_df[category].append(new_doc)

                        ctr += 1

                category_frequency[category] += 1
                total_documents += 1


                


In [83]:
classify()
vocabulary

defaultdict(list,
            {'business': ['Ad sales',
              'sales boost',
              'boost Time',
              'Time Warner',
              'Warner profit',
              'profit Quarterly',
              'Quarterly profits',
              'profits at',
              'at US',
              'US media',
              'media giant',
              'giant TimeWarner',
              'TimeWarner jumped',
              'jumped 76%',
              '76% to',
              'to $1.13bn',
              '$1.13bn (Â£600m)',
              '(Â£600m) for',
              'for the',
              'the three',
              'three months',
              'months to',
              'to December,',
              'December, from',
              'from $639m',
              '$639m year-earlier.',
              'year-earlier. The',
              'The firm,',
              'firm, which',
              'which is',
              'is now',
              'now one',
              'one of',
             

In [84]:
category_frequency

defaultdict(float,
            {'business': 510.0,
             'entertainment': 386.0,
             'politics': 417.0,
             'sport': 511.0,
             'tech': 401.0})

In [85]:
from math import log
logiprior = defaultdict(float)
for category in category_frequency:
    logiprior[category] = log(category_frequency[category] / total_documents)

logiprior

defaultdict(float,
            {'business': -1.4731014688819692,
             'entertainment': -1.7516748251355099,
             'politics': -1.6744259728015392,
             'sport': -1.471142604396636,
             'tech': -1.7135507672937715})

In [86]:
word_count = defaultdict(int)
words_vocabulary = set()

for category in vocabulary:
    for word in vocabulary[category]:
        word_count[word] += 1
        words_vocabulary.add(word)

word_count

defaultdict(int,
            {'Ad sales': 1,
             'sales boost': 1,
             'boost Time': 1,
             'Time Warner': 2,
             'Warner profit': 1,
             'profit Quarterly': 1,
             'Quarterly profits': 1,
             'profits at': 5,
             'at US': 10,
             'US media': 1,
             'media giant': 3,
             'giant TimeWarner': 1,
             'TimeWarner jumped': 1,
             'jumped 76%': 1,
             '76% to': 1,
             'to $1.13bn': 1,
             '$1.13bn (Â£600m)': 1,
             '(Â£600m) for': 1,
             'for the': 1411,
             'the three': 48,
             'three months': 57,
             'months to': 30,
             'to December,': 2,
             'December, from': 1,
             'from $639m': 1,
             '$639m year-earlier.': 1,
             'year-earlier. The': 1,
             'The firm,': 9,
             'firm, which': 7,
             'which is': 177,
             'is now': 110,
  

In [87]:
from collections import Counter
loglikelihood = defaultdict(int)

for category in vocabulary:
    counter = Counter(vocabulary[category])
    for word in words_vocabulary:
        loglikelihood[(word, category)] = log((counter[word] + 1) / (len(vocabulary[category]) + 1))

loglikelihood



defaultdict(int,
            {('Mariah Carey,', 'business'): -11.810820210098175,
             ('between 18', 'business'): -11.810820210098175,
             ('regulation was', 'business'): -11.11767302953823,
             ('identify land', 'business'): -11.810820210098175,
             ('worms appeared', 'business'): -11.810820210098175,
             ("Wilson's The", 'business'): -11.810820210098175,
             ('collapse would', 'business'): -11.11767302953823,
             ('colleagues Haisheng', 'business'): -11.810820210098175,
             ("LSE's chief", 'business'): -11.11767302953823,
             ('aside and', 'business'): -11.810820210098175,
             ('year after', 'business'): -9.412924937299804,
             ("or Hitler's", 'business'): -11.810820210098175,
             ('Their show', 'business'): -11.810820210098175,
             ('their strategies', 'business'): -11.810820210098175,
             ('Tonight at', 'business'): -11.810820210098175,
             ('days, 

In [88]:
def test(test_df, logiprior, loglikelihood, categories, words_vocabulary):
    ans = defaultdict(int)
    
    for category in categories:
        summ = logiprior[category]

        for word in test_df:
            if word in words_vocabulary:
                summ += loglikelihood[(word, category)]


        ans[category] = summ


    output = -float('inf')
    for category in ans:
        if ans[category] > output:
            best = category
            output = ans[category]

    return best





In [89]:
categories = []
for cat in vocabulary:
    categories.append(cat)

In [92]:
ctr = total = 0
for category in test_df:
    for doc in test_df[category]:
        output = test(doc, logiprior, loglikelihood, categories, words_vocabulary)
        if output == category:
            ctr += 1
        total += 1


print(f'Acurracy: {ctr / total * 100}%')

Acurracy: 96.62162162162163%
